In [14]:
import pymongo
from pymongo import MongoClient

import numpy as np

import getpass

import base64

# connecting to the database

In [10]:
client = MongoClient('localhost:27017')
db = client.arXivDB
db.users.count()

1

## logging in

In [86]:
while True:
    username = input('username: ').lower()
    user = list(db.users.find({'username': username}))
    if not user:
        print('the username doesnt exist, try again')
        user = None
    else:
        pin = base64.b64encode(bytes(str(getpass.getpass('pin: ')), encoding="UTF-8"))
        if not user[0]['pin']==pin:
            print('pin is incorrect, try again')
            user = None
        else:
            break

username: amir
pin: ········


In [87]:
%%time

num_papers_toshow = int(input('how many papers do you want to rate?: '))
print('=============================')
print('press y+enter if you like, otherwise just enter')
print('=============================')
for j, entry in enumerate(db.arXivfeeds.aggregate([{"$sample": {'size': num_papers_toshow}}])):
    like = input(str(num_papers_toshow-j)+'. '+entry['title'] + ': ')
    if like == 'y':
        db.likes.update_one({'paper_id': entry['_id']}, 
                            {'$set': {'paper_id': entry['_id'], 'user_id': user[0]['_id'], 'like':True}},
                           upsert=True)
    else:
        db.likes.update_one({'paper_id': entry['_id']}, 
                            {'$set': {'paper_id': entry['_id'], 'user_id': user[0]['_id'], 'like':False}},
                           upsert=True)

how many papers do you want to rate?: 500
press y+enter if you like, otherwise just enter
500. Generation and evaluation of entanglement between two multi-photon
  systems using single photon sources and linear optics: y
499. On the Relationship Between Complex Potentials and Strings of Projection
  Operators: 
498. Witness for a measure of genuine multipartite quantum discord for
  arbitrary N partite quantum state: 
497. Untouched aspects of the wave mechanics of a particle in one dimensional
  box: 
496. Complex Acquisition of the Fourier Transform Imaging of an Arbitrary
  Object: 
495. Security of quantum key distribution with multiphoton components: 
494. Improved phase gate reliability in systems with neutral Ising anyons: 
493. Spin squeezing of high-spin, spatially extended quantum fields: 
492. Angular Tunneling Effect: 
491. Quantum cryptography as a retrodiction problem: 
490. Bohm's Quantum Potential as an Internal Energy: 
489. Inequivalent representations of commutator o

In [85]:
mypapers = [d['paper_id'] for d in db.likes.find({'$and': [
                {'like':True}, {'user_id':user[0]['_id']}
            ]}, {'paper_id':1, '_id':0})]
for entry in db.arXivfeeds.find({'_id': {'$in': mypapers}}):
    print(entry['title'])

Teleportation via generalized measurements, and conclusive teleportation
Picturing Qubits in Phase Space
A consistent quantum model for continuous photodetection processes
A position and a time for the photon
Entangled Fock states for Robust Quantum Optical Metrology, Imaging, and
  Sensing
Strongly Incompatible Quantum Devices
Efficient high-dimensional entanglement imaging with a compressive
  sensing, double-pixel camera
Limits of reductionism and the measurement problem
Experimental determination of multipartite entanglement with incomplete
  information
Distinguishing Causality Principles
Negativity and steering: a stronger Peres conjecture
Analysis of the Christensen et al. Clauser-Horne (CH)-Inequality-Based
  Test of Local Realism
Complementary Decompositions and Unextendible Mutually Unbiased Bases
Quantum Cloning using Protective Measurement
Evaluation of entanglement measures by a single observable
A semidefinite programming upper bound of quantum capacity
Leggett--Garg ineq